In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
)
import torch






device = "auto"
model_path = "outputs_squad/merged_model"             # Path to the combined weights





bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True,
    )
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    trust_remote_code=True, 
    device_map=device, 
    torch_dtype=torch.bfloat16,
    # load_in_8bit=True,
    quantization_config=bnb_config if device == "auto" else None,
).eval()
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# Prompt should be in this style due to how the data was created
# prompt = "score: 100\n"
# prompt = "score: -10\n"
prompt = "#### Human: What's the capital of Australia?#### Assistant: "
# prompt = "I have an opinion\t\t"
# prompt = ""

limit = 128


inputs = tokenizer(prompt, return_tensors="pt")
if device != "cpu":
    inputs = inputs.to('cuda')
# del inputs['token_type_ids']
output = model.generate(**inputs, temperature=0.1, do_sample=True, top_p=0.95, top_k=60, max_new_tokens=limit-len(inputs["input_ids"]), pad_token_id=tokenizer.pad_token_id)
output = tokenizer.decode(output[0], skip_special_tokens=True)

# print(output)
print(output.split("#### Assistant:")[1].strip())